# Showcase of XGBoost

As the title says, this is only a showcase.  
For actual documentation, refer to the [xgboost documentation](https://xgboost.readthedocs.org/en/latest/).

In [1]:
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt
import toolbox as tb
import kaggleData as kD

In [2]:
bst = xgb.Booster({'nthread':16})
bst.load_model( "models/higgs.model.16step.depth3" )

In [6]:
mainFig = plt.figure(figsize = (16,9))
ax = mainFig.add_subplot(111)

#kwargs.update({'dpi': 1080})
xgb.plot_tree(bst, num_trees=0,ax=ax,dpi="270.0")

mainFig.savefig("tree.pdf",format="pdf",dpi=270)
plt.close("all")

Populating the interactive namespace from numpy and matplotlib
